In [ ]:
import os
import time
import sys
import platform
from Lib.YamlFrames import OpenYaml, GetWormName,Serial_Extraction_Data1

time_start = time.time()

workpath = os.getcwd()

yaml_path = os.path.join(workpath,'YamlFiles','20210516_2051_w1.yaml')


ListFile = OpenYaml(yaml_path)



wormname = GetWormName(yaml_path)



YamlData = Serial_Extraction_Data1(yaml_path)

time_end = time.time()
print('time cost:',time_end-time_start,'s')


In [ ]:
import pickle

# 写入文件
out_path = os.path.join(workpath,'YamlFiles',wormname+'.pkl')
with open(out_path,'wb') as output_pkl:
    pkl_str = pickle.dumps(YamlData)
    output_pkl.write(pkl_str)

In [ ]:
# 导入文件
from Lib.YamlFrames import YamlFrames
input_path = os.path.join(workpath,'YamlFiles',wormname+'.pkl')
with open(input_path,'rb') as input_pkl:
    YamlData2 = pickle.loads(input_pkl.read())
from Lib.YamlFrames import Get_Angle_Curve

centerline = YamlData2.Centerline
centerline_i = centerline[39567,:,:]
# 1722 1723 5962 39567
# worm_length,angle,curve,curvedatafiltered =Get_Angle_Curve(centerline1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a = '3.37033841185510	3.84443160222765	4.17242102459199	4.65563600168714	5.07442914892956	5.51192962853231	5.85353936923806	6.23246481907684	6.39417687230448	6.67059521454247	7.02871215588420	7.29541596379237	7.86665568677796	7.95782800547026	8.14368758822258	8.01652396817725	7.72491742957223	7.12693529406668	6.51106922461386	5.67115644970805	4.72191873209723	3.82272293583677	2.70268411399958	1.85901928554863	0.932136454234789	0.178057997185459	-0.567364123349694	-1.37422549286193	-2.15111785736816	-3.15316700495802	-4.24129434701814	-5.16026415969407	-6.33011574758513	-7.18045813153856	-8.11764036888127	-8.87282656273110	-9.53381099166493	-10.1560460059647	-10.7413127806968	-11.0057086435730	-11.1483338209488	-11.3556146996828	-11.1949889994973	-11.2073330753035	-10.9463639066619	-10.5897047518360	-10.1276916879160	-9.34263528978045	-8.41386890036867	-7.40992310575966	-6.27973389296286	-5.12137712830029	-3.94596129894350	-2.82005575233581	-1.82879001132495	-1.02982264106327	-0.475081490767604	-0.102916185087678	0.0193948183186565	0.0790896870631740	0.0440498276133282	0.0202920425098307	0.0335759918214196	0.0881393166704481	0.155761415252291	0.367878071523973	0.706207630389755	1.24598566618570	1.98414885799374	2.79250862586846	3.89559007856617	4.83556832556740	6.03267284845466	6.95120903217050	8.14844757491839	9.16203104685024	10.1250973826967	11.0005185798964	11.5796383715666	12.1745527423580	12.2945600381745	12.4392607403724	12.2734909194420	12.1972722605655	11.9104281127857	11.6471625177647	11.2560803076814	10.8153009445504	10.4691047507564	9.87997602680270	9.36921344191721	8.76522111485182	7.89093788437126	7.05045539252089	5.98503114967319	4.97131270443772	4.07956253779139	3.19922239750382	2.42430229581233	1.80045826457552'
cid = 9057
centerline = YamlData.Centerline[cid,:,:]
curvedatafiltered1 = YamlData.curvedatafiltered[cid,:]
 
curvedatafiltered= -curvedatafiltered1[::-1]


b = a.split('	')
c = np.zeros((1,100)).flatten()
for i in range(0,100):
    c[i] = float(b[i])
    

x = np.linspace(1,100,100,endpoint=True)
plt.figure()
plt.title('Compare matlab and python')
plt.plot(x,c[:],label='matlab curvedatafiltered')
plt.plot(x,curvedatafiltered[:],label='python curvedatafiltered')
plt.legend()
plt.xlabel('i')
plt.ylabel('curvedatafiltered')
plt.show()

plt.figure()
plt.title('Centerline')
plt.plot(centerline[0,:],centerline[1,:])
plt.show()

In [ ]:
import numpy as np
from csaps import csaps
from scipy import interpolate
import scipy.ndimage
worm_length = 0

numcurvepts = 100
proximity = 50
spline_p = 0.0005
flip=0
timefilter = 5
bodyfilter =10

centerline = YamlData.Centerline #列差分计算，相邻点做差分
centerline1 = centerline[20001,:,:]
df = np.diff(centerline1)
df1 = df*df
dfs = np.sqrt(np.dot([1,1],df1))
dft = dfs.reshape(1,len(dfs))
t0 = np.insert(dft,0,0)
t = np.cumsum(t0)
tt = t.reshape(1,len(t))
worm_length=worm_length+t[-1]

cv0 = csaps(t,centerline1,smooth = spline_p)
cv2 = cv0(t)
df2 = np.diff(cv2)

df2s = np.sqrt(np.dot([1,1],df2*df2))
dft = df2s.reshape(1,len(df2s))
splen = np.cumsum(np.insert(dft,0,0)).reshape(1,100)
# cv2 = np.interp(splen+0.00001*np.linspace(0,splen.size-1,splen.size,endpoint=True,dtype=float),cv2.transpose().reshape(100,2),(np.linspace(0,splen[0,-1]-1,102)).reshape(1,102))

cv2i0 = splen+0.00001*np.linspace(0,splen.size-1,splen.size,endpoint=True,dtype=float)
cv2i1 = cv2
cv2i2 = (np.linspace(0,splen[0,-1]-1,102)).reshape(1,102)
cv2i = interpolate.interp1d(cv2i0.reshape(100,),cv2i1,bounds_error=False)
# df*df
# t = cumsum([0,np.sqrt(np.array([1,1])*(df*df))])
newcv = np.array([cv2i(cv2i2)[0,0,:],cv2i(cv2i2)[1,0,:]]).reshape(2,102)

dfcv2= np.diff(newcv)
atdf2 = np.arctan2(-dfcv2[1,:],dfcv2[0,:])
angle = np.unwrap(atdf2)
curve = (np.unwrap(np.diff(angle))).reshape(1,100)

kernel = np.ones((timefilter,bodyfilter), np.float32)/(timefilter*bodyfilter)
curvedatafiltered = (scipy.ndimage.correlate(curve*100, kernel, mode='nearest')).flatten()
curvedatafiltered = -curvedatafiltered[::-1]
